# ETL Data Processor - DS2002 Project 1

Deliverable: Author a segment of an ETL pipeline that will ingest or process raw data. Submit a URL to a GitHub repository for your solution. In python you’ll need to know how to open files, iterate files, pattern match and output files.

Due date: Oct 31st, 11:59 PM

Benchmarks: data processor should be able to ingest a pre-defined data source and perform at least 3 of these operations:
1. Fetch / download / retrieve a remote data file by URL (API call like we did
in class), or ingest a local file that you have downloaded from
somewhere…like in a CSV format. Suggestions for remote data sources
are listed at the end of this document on our Github page as we went
through in class.
2. Convert the general format and data structure of the data source (from
JSON to CSV, from CSV to JSON, from JSON into a SQL database table,
etc. I want the option to convert any source to any target. So, if I get a
CSV as an input, I want the user to choose an output)
3. Modify the number of columns from the source to the destination,
reducing or adding columns so that you transform it with something
else…you can make up whatever it is…like date changes…or mash up
two columns…it’s up to you.
4. The converted (new) file should be written to disk (local file) or written to a
SQL database like SQL Lite
5. Generate a brief summary of the data file ingestion after it has processed
and output it to the user including:
     1. Number of records
     2. Number of columns
<br>The processor should produce informative errors should it be unable to complete
an operation. (Try / Catch with error messages, not file exists…just pick any
error.)

In [1]:
# imports
import json
import requests
import pandas as pd
import csv
from IPython.core.display import HTML
import sqlite3
from datetime import datetime
import numpy as np

In [6]:
print("Enter your client access token: ")
# mine is 'UUsrN0YdG7AF2g7G_YvYwfZnCf_xCeq71vywHCILsLuk4GZYuzfVHbiWJlHboKCc' feel free to use it for this 
client_access_token = input()

Enter your client access token: 
UUsrN0YdG7AF2g7G_YvYwfZnCf_xCeq71vywHCILsLuk4GZYuzfVHbiWJlHboKCc


In [51]:
print("Which artist would you like to search? ")
artist = input()

Which artist would you like to search? 
the chicks


In [52]:
# base url (genius)
search_url = f"http://api.genius.com/search?q={artist}&access_token={client_access_token}"
search_url

'http://api.genius.com/search?q=the chicks&access_token=UUsrN0YdG7AF2g7G_YvYwfZnCf_xCeq71vywHCILsLuk4GZYuzfVHbiWJlHboKCc'

In [53]:
def get_api_response(url, response_type):
    try:
        response = requests.get(url)
        response.raise_for_status()
    
    except requests.exceptions.HTTPError as errh:
        return "An Http Error occurred: " + repr(errh)
    except requests.exceptions.ConnectionError as errc:
        return "An Error Connecting to the API occurred: " + repr(errc)
    except requests.exceptions.Timeout as errt:
        return "A Timeout Error occurred: " + repr(errt)
    except requests.exceptions.RequestException as err:
        return "An Unknown Error occurred: " + repr(err)

    if response_type == 'json':
        result = json.dumps(response.json(), sort_keys=True, indent=4)
    else:
        result = "An unhandled error has occurred!"
        
    return result

In [54]:
r = requests.request("GET", search_url)
#r.content
get_api_response(search_url, json)
genius_json = r.json()
genius_json

{'meta': {'status': 200},
 'response': {'hits': [{'highlights': [],
    'index': 'song',
    'type': 'song',
    'result': {'annotation_count': 5,
     'api_path': '/songs/122323',
     'artist_names': 'The Chicks',
     'full_title': "Travelin' Soldier by\xa0The\xa0Chicks",
     'header_image_thumbnail_url': 'https://images.genius.com/aa6aa23a35a330ff54ad6b4f9a903970.300x300x1.jpg',
     'header_image_url': 'https://images.genius.com/aa6aa23a35a330ff54ad6b4f9a903970.1000x1000x1.jpg',
     'id': 122323,
     'language': 'en',
     'lyrics_owner_id': 132337,
     'lyrics_state': 'complete',
     'path': '/The-chicks-travelin-soldier-lyrics',
     'pyongs_count': 13,
     'relationships_index_url': 'https://genius.com/The-chicks-travelin-soldier-sample',
     'release_date_components': {'year': 2002, 'month': 7, 'day': 26},
     'release_date_for_display': 'July 26, 2002',
     'release_date_with_abbreviated_month_for_display': 'Jul. 26, 2002',
     'song_art_image_thumbnail_url': 'https

In [55]:
genius_json['response']['hits'][0]
# exploring

{'highlights': [],
 'index': 'song',
 'type': 'song',
 'result': {'annotation_count': 5,
  'api_path': '/songs/122323',
  'artist_names': 'The Chicks',
  'full_title': "Travelin' Soldier by\xa0The\xa0Chicks",
  'header_image_thumbnail_url': 'https://images.genius.com/aa6aa23a35a330ff54ad6b4f9a903970.300x300x1.jpg',
  'header_image_url': 'https://images.genius.com/aa6aa23a35a330ff54ad6b4f9a903970.1000x1000x1.jpg',
  'id': 122323,
  'language': 'en',
  'lyrics_owner_id': 132337,
  'lyrics_state': 'complete',
  'path': '/The-chicks-travelin-soldier-lyrics',
  'pyongs_count': 13,
  'relationships_index_url': 'https://genius.com/The-chicks-travelin-soldier-sample',
  'release_date_components': {'year': 2002, 'month': 7, 'day': 26},
  'release_date_for_display': 'July 26, 2002',
  'release_date_with_abbreviated_month_for_display': 'Jul. 26, 2002',
  'song_art_image_thumbnail_url': 'https://images.genius.com/ad9f3f046b17b05868529aefdb2549c7.300x300x1.jpg',
  'song_art_image_url': 'https://ima

In [56]:
hits = []
for i in genius_json['response']['hits']:
    hits.append(i['result']['title_with_featured'])

# get top five artist hits 
# exploring
for i in range(0, 5):
    print(hits[i])

Travelin' Soldier
Gaslighter
Not Ready to Make Nice
March March
Landslide


In [57]:
songs_list = []
for song in genius_json['response']['hits']:
    songs_list.append([song['result']['title_with_featured'], song['result']['stats']['pageviews'], song['result']['release_date_for_display'], song['result']['song_art_image_url']])

# create dataframe and show only the song title, number of page views, date released, and album cover url for the top five artist hits 
genius_df = pd.DataFrame(songs_list)
genius_df.columns = ['song title', 'page views', 'date released', 'album_cover_url']
genius_df.head(5)

,song title,page views,date released,album_cover_url
0,Travelin' Soldier,87424,"July 26, 2002",https://images.genius.com/ad9f3f046b17b0586852...
1,Gaslighter,101365,"March 4, 2020",https://images.genius.com/10068fde15aa8682656d...
2,Not Ready to Make Nice,58976,"March 24, 2006",https://images.genius.com/6782708e69d4b6354d61...
3,March March,57586,"June 25, 2020",https://images.genius.com/10068fde15aa8682656d...
4,Landslide,31199,"September 2, 2002",https://images.genius.com/5ac3dea65ba38fae3252...


In [58]:
hit_info = {'Song name':[], 'Page views': [], 'Date released':[]}
for i in genius_json['response']['hits'][:5]:
    hit_info['Song name'].append(i['result']['title'])
    hit_info['Page views'].append(i['result']['stats']['pageviews'])
    hit_info['Date released'].append(i['result']['release_date_for_display'])
hit_info
# create dictionary to prepare to send to .txt file

{'Song name': ['Travelin’ Soldier',
  'Gaslighter',
  'Not Ready to Make Nice',
  'March March',
  'Landslide'],
 'Page views': [87424, 101365, 58976, 57586, 31199],
 'Date released': ['July 26, 2002',
  'March 4, 2020',
  'March 24, 2006',
  'June 25, 2020',
  'September 2, 2002']}

In [59]:
def get_html(url):
    # function to get HTML from a url 
    image_html = f"<img src='{url}' width='100px'>"
    return image_html

In [60]:
genius_df['album cover'] = genius_df['album_cover_url'].apply(get_html)
# send url to get_html function
genius_df.head(5)

,song title,page views,date released,album_cover_url,album cover
0,Travelin' Soldier,87424,"July 26, 2002",https://images.genius.com/ad9f3f046b17b0586852...,<img src='https://images.genius.com/ad9f3f046b...
1,Gaslighter,101365,"March 4, 2020",https://images.genius.com/10068fde15aa8682656d...,<img src='https://images.genius.com/10068fde15...
2,Not Ready to Make Nice,58976,"March 24, 2006",https://images.genius.com/6782708e69d4b6354d61...,<img src='https://images.genius.com/6782708e69...
3,March March,57586,"June 25, 2020",https://images.genius.com/10068fde15aa8682656d...,<img src='https://images.genius.com/10068fde15...
4,Landslide,31199,"September 2, 2002",https://images.genius.com/5ac3dea65ba38fae3252...,<img src='https://images.genius.com/5ac3dea65b...


In [61]:
HTML(genius_df[['song title', 'album cover', 'date released', 'page views']].head(5).to_html(escape = False))
# turn HTML into an actual image

,song title,album cover,date released,page views
0,Travelin' Soldier,,"July 26, 2002",87424
1,Gaslighter,,"March 4, 2020",101365
2,Not Ready to Make Nice,,"March 24, 2006",58976
3,March March,,"June 25, 2020",57586
4,Landslide,,"September 2, 2002",31199


In [62]:
new_df = genius_df[['song title', 'page views', 'date released']]                                                          
new_genius_df = new_df[:5]
new_genius_df
# getting only the stuff I want to export

,song title,page views,date released
0,Travelin' Soldier,87424,"July 26, 2002"
1,Gaslighter,101365,"March 4, 2020"
2,Not Ready to Make Nice,58976,"March 24, 2006"
3,March March,57586,"June 25, 2020"
4,Landslide,31199,"September 2, 2002"


In [63]:
date_components = []
for i in genius_json['response']['hits'][:5]:
    date_components.append(i['result']['release_date_components'])
date_components
# get the date components (not the date with the written out month)

[{'year': 2002, 'month': 7, 'day': 26},
 {'year': 2020, 'month': 3, 'day': 4},
 {'year': 2006, 'month': 3, 'day': 24},
 {'year': 2020, 'month': 6, 'day': 25},
 {'year': 2002, 'month': 9, 'day': 2}]

In [64]:
hit_date_components = []
for i in range(0, 5):
    for value in date_components[i].values():
        hit_date_components.append(value)

components = np.array_split(hit_date_components, 5)
# get just values

In [65]:
type(components[1][1])
# what type is each component?

numpy.int64

In [66]:
new_components = []
for i in range(0, 5):
    new_components.append(np.array2string(components[i]))
    # array --> string

new_components2 = []
for item in new_components:
    new_components2.append(item.split())
    # split by commas to facilitate zero padding 

In [67]:
date_objects = []
for i in range(0, len(new_components2)):
    year = new_components2[i][0][1:]
    month = new_components2[i][1]
    day = new_components2[i][2][:-1]
    date_str = month + "-" + day + "-" + year
    date_objects.append(datetime.strptime(date_str, '%m-%d-%Y').date())

date_objects
# list of dates in datetime format to do math on them

[datetime.date(2002, 7, 26),
 datetime.date(2020, 3, 4),
 datetime.date(2006, 3, 24),
 datetime.date(2020, 6, 25),
 datetime.date(2002, 9, 2)]

In [68]:
today = datetime.now().date()
# get todays date

In [69]:
difference_in_days = []
for i in range(0, len(date_objects)):
    difference_in_days.append((today - date_objects[i]).days)
    
difference_in_days # get number of days ago that the song was dropped

[7395, 964, 6058, 851, 7357]

In [70]:
average_views = []
for i in range(0, len(new_genius_df)):
    average_views.append(new_genius_df['page views'][i] / difference_in_days[i])

new_genius_df['days since release'] = difference_in_days
new_genius_df['views / days since release'] = average_views
new_genius_df.loc[:, ["song title","page views","date released","days since release", "views / days since release"]]
# add new stuff to new_genius_df

<ipython-input-70-94f6c0f8dd38>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_genius_df['days since release'] = difference_in_days
<ipython-input-70-94f6c0f8dd38>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_genius_df['views / days since release'] = average_views


,song title,page views,date released,days since release,views / days since release
0,Travelin' Soldier,87424,"July 26, 2002",7395,11.822042
1,Gaslighter,101365,"March 4, 2020",964,105.150415
2,Not Ready to Make Nice,58976,"March 24, 2006",6058,9.735226
3,March March,57586,"June 25, 2020",851,67.668625
4,Landslide,31199,"September 2, 2002",7357,4.240723


In [71]:
print("What format output would you like? Options: csv, sql, json" )
output_format = input()

What format output would you like? Options: csv, sql, json
csv


In [72]:
if output_format == 'csv':
    new_genius_df.to_csv('hits.csv', encoding = 'utf-8', index = False)
elif output_format == 'sql':
    con = sqlite3.connect("hits.db")
    cur = con.cursor()
    cur.execute("CREATE TABLE IF NOT EXISTS hits(song_title TEXT, page_views INT, date_released TEXT)")
    con.commit()
    new_genius_df.to_sql('hits', con, if_exists = 'replace', index = False)
    cur.execute('''SELECT * FROM hits''')
    for row in cur.fetchall():
        print(row)
elif output_format == 'json':
    result = new_genius_df.to_json(orient = "split")
    parsed = json.loads(result)
    json_object = json.dumps(parsed, indent = 4)
    with open("hits_as_json.txt", "w") as outfile:
        outfile.write(json_object)
else:
    print("Error: please go back and enter a valid output format!")

## DOCUMENTATION: HOW TO USE THIS DATA PROCESSOR 

The purpose of this data processor is to return the top five hits from a music artist based on page views via the Genius API. First, when prompted, please input your client access token to the API, and then enter an artist you'd like to retrieve information about. When this notebook is run, it will return a DataFrame that contains your artist's top five hits' song titles, dates released, page views, number of days since release, and number of page views divided by the days since release. (Note: while rare, you may have issues with some artists because they are either too small or certain things are missing from their Genius profile- this may happen if the artist is older and data is not available for hit release date, for example) At this point, when prompted, please input the output format you would like for the DataFrame. 